# Import Libraries

In [11]:
from GenerateSyntheticData import *
from Clustering import clusterKMeansBase
from ClusteredMeanDecreaseAccuracy import clustered_feature_importance_MDA

import sklearn.ensemble as Ensemble
import sklearn.tree as Tree
import pandas as pd
import numpy as np

# Generate Synthetic Test Data

In [12]:
# def get_time_series_test_dataset(
#     n_informative: int = 10,  # number of informative features
#     n_redundant: int = 10,  # number of redundant features
#     n_noise: int = 20,
#     n_samples: int = 10000,  # number of sample to generate
#     n_time_steps: int = 7, # Two business years
#     random_state: int = 0,  # random state
#     sigma_std: float = 0.0,  # standard deviation of generation
# ):
#     np.random.seed(random_state)

#     # Generate informative features
#     informative_features = np.random.randn(n_samples, n_informative, n_time_steps)

#     # Generate redundant features (linear form of informative features)
#     i = np.random.choice(range(n_informative), size=n_redundant)
#     print(i)

#     linear_redundant_features = np.zeros((n_samples, n_redundant, n_time_steps))
#     linear_redundant_features_from_ = []
#     for k, j in enumerate(i):
#         linear_redundant_features[:, k, :] = informative_features[:, j, :] + np.random.normal(size=(n_samples, n_time_steps)) * sigma_std
#         linear_redundant_features_from_.append(j)


#     i = np.random.choice(range(n_informative), size=n_redundant)
#     print(i)

#     quadratic_redundant_features = np.zeros((n_samples, n_redundant, n_time_steps))
#     quadratic_redundant_features_from_ = []
#     for k, j in enumerate(i):
#         quadratic_redundant_features[:, k, :] = informative_features[:, j, :] ** 2 + np.random.normal(size=(n_samples, n_time_steps)) * sigma_std
#         quadratic_redundant_features_from_.append(j)

        
#     # Generate noise features
#     noise_features = np.random.randn(n_samples, n_noise, n_time_steps) * sigma_std

#     # Concatenate all features
#     X = np.concatenate([
#         informative_features,
#         linear_redundant_features,
#         quadratic_redundant_features,
#         noise_features
#         ],
#         axis=1)

#     columns = [f"I{i}_lag{j}" for i in range(n_informative) for j in range(n_time_steps)]
#     columns += [f"R_{i}_lag{j}_linear_from_I{k}" for i, k in enumerate(linear_redundant_features_from_) for j in range(n_time_steps)]
#     columns += [f"R_{i}_lag{j}_quadratic_from_I{k}" for i, k in enumerate(quadratic_redundant_features_from_) for j in range(n_time_steps)]
#     columns += [f"N{i}_lag{j}" for i in range(n_noise) for j in range(n_time_steps)]

#     X = X.reshape((n_samples, -1))
#     X = pd.DataFrame(X, columns=columns)

#     # Generate labels (y) based on informative features
#     y = np.mean(np.sum(informative_features, axis=2), axis=1) > 0
#     y = y.astype(int)
#     y = pd.Series(y)

#     return X, y

# n_time_steps = 3
# sigma_std = 0.25

# # X, y = get_test_dataset(
# #     n_informative=10, 
# #     n_redundant=10,
# #     n_noise=10,
# #     n_samples=1000,
# #     sigma_std=0.5
# # )

# X, y = get_time_series_test_dataset(
#     n_informative=5,
#     n_redundant=5,
#     n_noise=5,
#     n_time_steps=n_time_steps,
#     n_samples=1000,
#     sigma_std=sigma_std
# )

In [13]:
X, y = get_test_dataset(
    n_features=40,
    n_informative=5, 
    n_redundant=30,
    n_samples=10000,
    sigma_std=1
)

# Fit Clustered MDA

## Clustering

In [14]:
import warnings
warnings.filterwarnings('ignore')

corr0, clusters, silh = clusterKMeansBase(
    X.corr(),
    numberClusters=round(len(X.columns) * 0.95),
    iterations=20);

for k, v in clusters.items():
    print(k, v)

0 ['I_3', 'R_2', 'R_3', 'R_4', 'R_6', 'R_20', 'R_22', 'R_25', 'R_28', 'R_29']
1 ['I_0', 'R_1', 'R_9', 'R_10', 'R_14', 'R_17', 'R_21', 'R_23', 'R_26']
2 ['N_0', 'N_1', 'N_2', 'N_3', 'N_4']
3 ['I_4', 'R_0', 'R_8', 'R_11', 'R_19']
4 ['I_2', 'R_7', 'R_12', 'R_24']
5 ['I_1', 'R_5', 'R_13', 'R_15', 'R_16', 'R_18', 'R_27']


## Fit

In [4]:
import warnings
warnings.filterwarnings('ignore')

classifier = Tree.DecisionTreeClassifier(
    criterion="entropy",
    max_features=1,
    class_weight="balanced",
    min_weight_fraction_leaf=0)

classifier = Ensemble.BaggingClassifier(
    base_estimator=classifier,
    n_estimators=1000,
    max_features=1.0,
    max_samples=1.0,
    oob_score=False)

n_splits = 10

result = clustered_feature_importance_MDA(
    classifier, 
    X, y,
    clusters,
    n_splits
)

fold 0 start ...
fold 1 start ...
fold 2 start ...
fold 3 start ...
fold 4 start ...
fold 5 start ...
fold 6 start ...
fold 7 start ...
fold 8 start ...


In [6]:
result = result.sort_values("Mean", ascending=False)
result

,Mean,StandardDeviation
C_3,0.221676,0.074736
C_0,0.180259,0.021679
C_1,0.170694,0.056375
C_2,0.165219,0.043435
C_4,0.141058,0.043428
C_5,0.000448,0.001263


# Plot Results

In [7]:
import plotly.express as px

result["ClusterIndex"] = result.index
fig = px.bar(
    result,
    x="Mean", 
    y="ClusterIndex",
    error_x="StandardDeviation"    
)

fig.update_layout(
    title="Clustered MDA Results",
    xaxis_title="Feature Importance",
    yaxis_title="Cluster Index",
    template="plotly_dark",
    width=800, height=1200,
    # legend_title="Legend Title",
    # font=dict(
    #     family="Courier New, monospace",
    #     size=18,
    #     color="RebeccaPurple"
    # )
)

fig.show()

# Save Results

In [8]:
fig.write_image("./Figs/clustered_MDA_results.png",format='png',engine='kaleido')